#### Lesson 5: Coding and Math Problem Solving

The python version of this lesson is Coding and Financial Analysis. Because of lacking financial and plotting library in dotnet, it is very challenging for GPT to analyze financial data using C# as programming language. So I change the topic to Coding and Math Problem Solving.

#### Setup

In [1]:
#r "nuget:AutoGen"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;
using System.IO;
using System.Reactive.Linq;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-3.5-turbo";
var openaiClient = new OpenAIClient(openAIKey);


Installed Packages AutoGen, 0.0.16

Agent with code executor configuration. Because we are in dotnet interactive notebook, we can simply using the dotnet interactive kernel to execute the code.

The code below retreive the csharp code between ```csharp and ``` and send it to dotnet interactive kernel to execute.

In [2]:
var codeBlockPrefix = "```csharp";
var codeBlockSuffix = "```";
var codeExecutorAgent = new DefaultReplyAgent(
    name: "code_executor_agent",
    defaultReply: "no code to execute")
    .RegisterMiddleware(async (msgs, option, innerAgent, ct) =>
        {
            var lastMessage = msgs.LastOrDefault();
            if (lastMessage == null || lastMessage.GetContent() is null)
            {
                return await innerAgent.GenerateReplyAsync(msgs, option, ct);
            }

            // retrieve all code blocks from last message
            var codeBlocks = lastMessage.GetContent()!.Split(new[] { codeBlockPrefix }, StringSplitOptions.RemoveEmptyEntries);
            if (codeBlocks.Length <= 0)
            {
                return await innerAgent.GenerateReplyAsync(msgs, option, ct);
            }

            // run code blocks
            var result = new StringBuilder();
            var i = 0;
            result.AppendLine(@$"// [DOTNET_CODE_BLOCK_EXECUTION]");
            foreach (var codeBlock in codeBlocks)
            {
                var codeBlockIndex = codeBlock.IndexOf(codeBlockSuffix);

                if (codeBlockIndex == -1)
                {
                    continue;
                }

                // remove code block suffix
                var code = codeBlock.Substring(0, codeBlockIndex).Trim();

                if (code.Length == 0)
                {
                    continue;
                }

                var events = new List<DisplayEvent>();
                Kernel.Current.RootKernel.KernelEvents.OfType<DisplayEvent>().Subscribe(e => events.Add(e));
                var kernelCommandResult = await Kernel.Current.RootKernel.SubmitCodeAsync(code);
                var displayValues = events.Where(x => x is StandardErrorValueProduced || x is StandardOutputValueProduced || x is ReturnValueProduced)
                    .SelectMany(x => (x as DisplayEvent)!.FormattedValues);
                
                if (displayValues is null || displayValues.Count() == 0)
                {
                    return new TextMessage(Role.Assistant, "No output", from: innerAgent.Name);
                }

                var codeResult = string.Join("\n", displayValues.Select(x => x.Value));
                result.AppendLine(@$"### Executing result for code block {i++}");
                result.AppendLine(codeResult);
                result.AppendLine("### End of executing result ###");
            }

            return new TextMessage(Role.Assistant, result.ToString(), from: innerAgent.Name);
        })
    .RegisterPrintMessage();

Agent with dotnet coding writing capability

In [3]:
var coderAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "code_writer_agent",
    modelName: openAIModel,
    systemMessage: """
    You act as dotnet coder, you write dotnet code to resolve task. Once you finish writing code, ask runner to run the code for you.

    Here're some rules to follow on writing dotnet code:
    - put code between ```csharp and ```
    - When creating http client, use `var httpClient = new HttpClient()`. Don't use `using var httpClient = new HttpClient()` because it will cause error when running the code.
    - Try to use `var` instead of explicit type.
    - Try avoid using external library, use .NET Core library instead.
    - Use top level statement to write code.
    - Always print out the result to console. Don't write code that doesn't print out anything.
    
    If you need to install nuget packages, put nuget packages in the following format:
    ```nuget
    nuget_package_name
    ```
    
    If your code is incorrect, Fix the error and send the code again.
    Once the task is resolved, say 'task completed' to finish the task.
    """,
    temperature: 0.4f)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

The task!

In [4]:
var task = """
    calculate the 39th fibonacci number and save the result to result.txt
    """;

Start the conversation to resolve the task.

In [5]:
var chatHistory = new List<IMessage>()
{
    new TextMessage(Role.Assistant, task, from: codeExecutorAgent.Name),
};
var maxRoundLeft = 10;

while (maxRoundLeft > 0)
{
    var reply = await coderAgent.SendAsync(
        receiver: codeExecutorAgent,
        chatHistory: chatHistory,
        maxRound: 1);

    if (reply.Last().GetContent()?.ToLower().Contains("task completed") == true)
    {
        break;
    }
    else
    {
        chatHistory.Add(reply.Last());
    }
    maxRoundLeft--;
}

from: code_writer_agent
```csharp
var n = 39;
var a = 0;
var b = 1;

for (var i = 0; i < n; i++)
{
    var temp = a;
    a = b;
    b = temp + b;
}

System.IO.File.WriteAllText("result.txt", b.ToString());
Console.WriteLine($"The 39th Fibonacci number is: {b}");
``` 

The 39th Fibonacci number is: 102334155
TextMessage from code_executor_agent
--------------------
// [DOTNET_CODE_BLOCK_EXECUTION]
### Executing result for code block 0
The 39th Fibonacci number is: 102334155

### End of executing result ###

--------------------

from: code_writer_agent
task completed

